In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .appName("Day15_1")\
        .getOrCreate()


In [ ]:

data = [
    ("O001","Amit","Hyderabad","Spice Hub","Indian",450,35,"UPI","Delivered"),
    ("O002","Neha","Bangalore","Pizza Town","Italian",650,40,"Card","Delivered"),
    ("O003","Rahul","Delhi","Burger Zone","American",520,30,"Cash","Delivered"),
    ("O004","Pooja","Mumbai","Sushi Bar","Japanese",1200,55,"UPI","Cancelled"),
    ("O005","Arjun","Chennai","Curry Leaf","Indian",380,28,"UPI","Delivered"),
    ("O006","Sneha","Hyderabad","Pasta Street","Italian",700,45,"Card","Delivered"),
    ("O007","Karan","Delhi","Taco Bell","Mexican",540,33,"UPI","Delivered"),
    ("O008","Riya","Bangalore","Dragon Bowl","Chinese",600,38,"Wallet","Delivered"),
    ("O009","Vikas","Mumbai","BBQ Nation","Indian",1500,60,"Card","Delivered"),
    ("O010","Anjali","Chennai","Burger Zone","American",480,32,"Cash","Delivered"),
    ("O011","Farhan","Delhi","Biryani House","Indian",520,36,"UPI","Delivered"),
    ("O012","Megha","Hyderabad","Sushi Bar","Japanese",1100,58,"Card","Cancelled"),
    ("O013","Suresh","Bangalore","Curry Leaf","Indian",420,29,"UPI","Delivered"),
    ("O014","Divya","Mumbai","Pizza Town","Italian",780,42,"Wallet","Delivered"),
    ("O015","Nikhil","Delhi","Pasta Street","Italian",690,47,"UPI","Delivered"),
    ("O016","Kavya","Chennai","Dragon Bowl","Chinese",560,34,"UPI","Delivered"),
    ("O017","Rohit","Hyderabad","BBQ Nation","Indian",1400,62,"Card","Delivered"),
    ("O018","Simran","Bangalore","Burger Zone","American",510,31,"Cash","Delivered"),
    ("O019","Ayesha","Mumbai","Taco Bell","Mexican",570,35,"UPI","Delivered"),
    ("O020","Manish","Delhi","Curry Leaf","Indian",390,27,"Wallet","Delivered"),
    ("O021","Priya","Hyderabad","Pizza Town","Italian",720,41,"Card","Delivered"),
    ("O022","Yash","Chennai","Sushi Bar","Japanese",1150,57,"UPI","Delivered"),
    ("O023","Naina","Bangalore","Pasta Street","Italian",680,44,"UPI","Delivered"),
    ("O024","Sameer","Mumbai","Dragon Bowl","Chinese",610,39,"Wallet","Delivered"),
    ("O025","Ritika","Delhi","Burger Zone","American",500,30,"Cash","Delivered"),
    ("O026","Gopal","Hyderabad","Curry Leaf","Indian",410,28,"UPI","Delivered"),
    ("O027","Tina","Bangalore","Pizza Town","Italian",760,43,"Card","Delivered"),
    ("O028","Irfan","Mumbai","BBQ Nation","Indian",1550,65,"Card","Delivered"),
    ("O029","Sahil","Chennai","Taco Bell","Mexican",590,37,"UPI","Delivered"),
    ("O030","Lavanya","Delhi","Dragon Bowl","Chinese",630,40,"Wallet","Delivered"),
    ("O031","Deepak","Hyderabad","Burger Zone","American",520,33,"Cash","Delivered"),
    ("O032","Shweta","Bangalore","Curry Leaf","Indian",450,31,"UPI","Delivered"),
    ("O033","Aman","Mumbai","Pizza Town","Italian",810,46,"Card","Delivered"),
    ("O034","Rekha","Chennai","Pasta Street","Italian",700,45,"UPI","Delivered"),
    ("O035","Zubin","Delhi","BBQ Nation","Indian",1480,63,"Card","Delivered"),
    ("O036","Pallavi","Hyderabad","Dragon Bowl","Chinese",580,36,"Wallet","Delivered"),
    ("O037","Naveen","Bangalore","Taco Bell","Mexican",560,34,"UPI","Delivered"),
    ("O038","Sonia","Mumbai","Sushi Bar","Japanese",1180,59,"Card","Delivered"),
    ("O039","Harish","Chennai","Burger Zone","American",490,29,"Cash","Delivered"),
    ("O040","Kriti","Delhi","Curry Leaf","Indian",420,26,"UPI","Delivered")
]
columns = [
    "order_id","customer_name","city","restaurant","cuisine",
    "order_amount","delivery_time_minutes","payment_mode","order_status"
]
df = spark.createDataFrame(data, columns)
df.show()
df.printSchema()





#Exercise 1 - Write the full dataset to CSV with header enabled.

In [29]:
import csv

with open ("orders.csv" , "w",newline ='') as f:
  writer = csv.writer(f)
  writer.writerow(columns)
  writer.writerows(data)


In [ ]:
df_csv = spark.read\
          .option("header",True)\
          .option("inferschema",True)\
          .csv("orders.csv")

df_csv.show(truncate = False)
df.printSchema()

#Exercise 2 - Read the CSV back and filter:
order_amount > 700

In [ ]:
import pyspark.sql.functions as F

df_csv = spark.read\
          .option("header",True)\
          .option("inferschema",True)\
          .csv("orders.csv")

df_filtered = df_csv.filter(F.col("order_amount") > 700)
df_filtered.show(truncate = False)

#Exercise 3 - From CSV, show only:
order_id
city
cuisine
order_amount

In [ ]:
df_selected = df_csv.select("order_id","city","cuisine","order_amount")
df_selected.show()

#Exercise 4 - Sort orders by delivery_time_minutes descending and write result to CSV.

In [42]:
from pyspark.sql import functions as F
df_sorted = df_csv.orderBy(F.col("delivery_time_minutes").desc())

#Exercise 5 - Write only “Delivered” orders to JSON.

In [ ]:
import pyspark.sql.functions as F
df_delivered = df.filter(
    F.lower(F.trim(F.col("order_status"))) == F.lit("delivered")
)
# Write to JSON (folder): delivered_orders_json/
(
    df_delivered
    .coalesce(1)                 # optional: single JSON part file
    .write
    .mode("overwrite")           # overwrite if the folder exists
    .json("delivered_orders_json")
)
print("Delivered count:", df_delivered.count())
df_delivered.show(10, truncate=False)

#Exercise 6
Read JSON and filter:
city = "Mumbai"
payment_mode = "Card"

In [ ]:
import pyspark.sql.functions as F
df_filter = df.filter((F.lower(F.trim(F.col("city"))) == F.lit("mumbai")) & (F.lower(F.trim(F.col("payment_mode"))) == F.lit("card")) )
df_filter.select(
"order_id", "customer_name", "city", "restaurant",
    "cuisine", "order_amount", "delivery_time_minutes",
    "payment_mode", "order_status"
).orderBy(F.col("order_amount").desc()).show(truncate=False)
print("Matching rows:", df_filtered.count())

#Exercise 7 - Add a column:
delivery_category
Logic:
delivery_time_minutes > 45 → "Late"
else → "OnTime"

In [47]:
import pyspark.sql.functions as F
df_1 = df.withColumn("delivery_category" , F.when(F.col("delivery_time_minutes") > 45, F.lit("Late")).otherwise(F.lit("OnTime")))
(
    df_1
    .coalesce(1)                 # optional: single JSON part file
    .write
    .mode("overwrite")
    .json("orders_with_delivery_category_json")
)

#Exercise 8 - Force JSON output to a single partition and observe number of files created.

In [ ]:
df_base = df

(
    df_base.
    coalesce(1).
    write.
    mode("overwrite").
    json("single_partition")
)
df_base.show()

#Exercise 9 - Convert full dataset to Parquet.
Output:
orders_parquet/

In [50]:
df_parquet = df.write.mode("overwrite").parquet("orders.parquet")


#Exercise 10 - Read Parquet and filter:
cuisine = "Indian"
order_amount > 500

In [ ]:

from pyspark.sql import functions as F
df_parquet = spark.read.parquet("orders.parquet")
df_parquet = df_parquet.withColumn("order_amount", F.col("order_amount").cast("int"))
df_filtered = df_parquet.filter(
    (F.col("cuisine") == "Indian") & (F.col("order_amount") > 500)
)
df_filtered.select(
    "order_id", "customer_name", "city", "restaurant",
    "cuisine", "order_amount", "delivery_time_minutes",
    "payment_mode", "order_status"
).orderBy(F.col("order_amount").desc()).show(truncate=False)
print("Matching row count:", df_filtered.count())


#Exercise 11 - Sort Parquet data by order_amount descending and write top 10 orders back to Parquet.

In [ ]:
top10_df=df_parquet.orderBy("order_amount",ascending=False).limit(10)
top10_df.write.mode("overwrite").parquet("top10_orders_parquet")
top10_df.show()

#Exercise 12 - Compare storage size of:
CSV
JSON
Parquet
Answer:
Which is smallest?
Why?

#Exercise 13 - Convert:
CSV → Parquet
JSON → Parquet